In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import os
from sklearn.impute import KNNImputer
import haversine as hs
from glob import glob
import pandas as pd
from tqdm import tqdm

In [ ]:
#columns name

In [2]:
col=['ID','Delivery_person_ID','Delivery_person_Age','Delivery_person_Ratings','Restaurant_latitude','Restaurant_longitude',
'Delivery_location_latitude','Delivery_location_longitude','Order_Date','Time_Orderd','Time_Order_picked',
'Weather_conditions','Road_traffic_density','Vehicle_condition','Type_of_order','Type_of_vehicle','multiple_deliveries',
'Festival','City','Time_taken (min)']

In [ ]:
#Importing file

In [3]:

train=pd.concat((pd.read_csv(file,delimiter='\s{2,}',
        header=None,usecols=[1],skipfooter=1).T for file in tqdm(glob('dataset\\train\\*'))),
          ignore_index=True)
train.columns=col

100%|██████████| 45593/45593 [18:11<00:00, 41.77it/s]


In [4]:
train.head(5)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0x4607,INDORES13DEL02,37.000000,4.900000,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30,11:45,Sunny,High,2,Snack,motorcycle,0.000000,No,Urban,24.000000
1,0xb379,BANGRES18DEL02,34.000000,4.500000,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45,19:50,Stormy,Jam,2,Snack,scooter,1.000000,No,Metropolitian,33.000000
2,0x5d6d,BANGRES19DEL01,23.000000,4.400000,12.914264,77.678400,12.924264,77.688400,19-03-2022,8:30,8:45,Sandstorms,Low,0,Drinks,motorcycle,1.000000,No,Urban,26.000000
3,0x7a6a,COIMBRES13DEL02,38.000000,4.700000,11.003669,76.976494,11.053669,77.026494,05-04-2022,17:60,18:10,Sunny,Medium,0,Buffet,motorcycle,1.000000,No,Metropolitian,21.000000
4,0x70a2,CHENRES12DEL01,32.000000,4.600000,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30,13:45,Cloudy,High,1,Snack,scooter,1.000000,No,Metropolitian,30.000000


In [ ]:
#importing test data

In [5]:

test=pd.concat((pd.read_csv(file,delimiter='\s{2,}',
        header=None,usecols=[1],skipfooter=1).T for file in tqdm(glob('dataset\\test\\*'))),
          ignore_index=True)
test.columns=col[:-1]

100%|██████████| 11399/11399 [04:55<00:00, 38.52it/s]


In [6]:
test.head(5)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City
0,0x2318,COIMBRES13DEL01,NaN,NaN,11.003669,76.976494,11.043669,77.016494,30-03-2022,NaN,15:05,NaN,NaN,3,Drinks,electric_scooter,1.000000,No,Metropolitian
1,0x3474,BANGRES15DEL01,28.000000,4.600000,12.975377,77.696664,13.085377,77.806664,29-03-2022,20:30,20:35,Windy,Jam,0,Snack,motorcycle,1.000000,No,Metropolitian
2,0x9420,JAPRES09DEL03,23.000000,4.500000,26.911378,75.789034,27.001378,75.879034,10-03-2022,19:35,19:45,Stormy,Jam,0,Drinks,motorcycle,1.000000,No,Metropolitian
3,0x72ee,JAPRES07DEL03,21.000000,4.800000,26.766536,75.837333,26.856536,75.927333,02-04-2022,17:15,17:20,Fog,Medium,1,Meal,scooter,1.000000,No,Metropolitian
4,0xa759,CHENRES19DEL01,31.000000,4.600000,12.986047,80.218114,13.096047,80.328114,27-03-2022,18:25,18:40,Sunny,Medium,2,Drinks,scooter,1.000000,No,Metropolitian


In [7]:
id=test['ID']

In [8]:
test['Time_taken (min)']=1
train['type']='train'
test['type']='test'
Df=pd.concat([train,test])

In [ ]:
#converting categorical value to continuous value

In [9]:
def mapping(df):
    Df['Weather_conditions']=Df['Weather_conditions'].map({'Sunny':0, 'Stormy':1, 'Sandstorms':2, 'Cloudy':3, 'Fog':4, 
                                                                'Windy':5})
    Df['Road_traffic_density']=Df['Road_traffic_density'].map({'High':2, 'Jam':3, 'Low':0, 'Medium':1})
    Df['Type_of_order']=Df['Type_of_order'].map({'Snack':0, 'Drinks':1, 'Buffet':2, 'Meal':3})
    Df['Type_of_vehicle']=Df['Type_of_vehicle'].map({'motorcycle':0, 'scooter':1, 'electric_scooter':2, 'bicycle':3})
    Df['Festival']=Df['Festival'].map({"No":0,'Yes':1})
    Df['City']=Df["City"].map({'Urban':1, 'Metropolitian':0, 'Semi-Urban':2})

In [10]:
mapping(Df)

In [ ]:
#converting some columns to float

In [11]:
Df['Restaurant_latitude']=Df['Restaurant_latitude'].astype(float)
Df['Restaurant_longitude']=Df['Restaurant_longitude'].astype(float)
Df['Delivery_location_latitude']=Df['Delivery_location_latitude'].astype(float)
Df['Delivery_location_longitude']=Df['Delivery_location_longitude'].astype(float)

In [ ]:
#Getting weekday from date 

In [12]:
Df['Order_Date']=pd.to_datetime(train.Order_Date)
Df['weekday']=Df['Order_Date'].dt.weekday

In [ ]:
#converting some negative value to absolute integer

In [13]:
Df['Restaurant_latitude']=Df['Restaurant_latitude'].abs()
Df['Restaurant_longitude']=Df['Restaurant_longitude'].abs()

In [ ]:
#some latitude and longitude are 0 or 1 so converting to na

In [14]:
Df['Restaurant_latitude']=Df['Restaurant_latitude'].mask(Df['Restaurant_latitude']<1)
Df['Restaurant_longitude']=Df['Restaurant_longitude'].mask(Df['Restaurant_longitude']<1)
Df['Delivery_location_latitude']=Df['Delivery_location_latitude'].mask(Df['Delivery_location_latitude']<1)
Df['Delivery_location_longitude']=Df['Delivery_location_longitude'].mask(Df['Delivery_location_longitude']<1)

In [ ]:
#seperatig all rows which has nan value 

In [15]:
na_data=Df[Df.isna().any(axis=1)]

In [16]:
Df.dropna(inplace=True)

In [ ]:
#creating a column which has duration between the time_order and picked by delivery person

In [17]:
def for_pick_dur(train):
    train['order_hour']=train['Time_Orderd'].str.split(':').str[0]
    train['order_min']=train['Time_Orderd'].str.split(':').str[1]
    train['order_picked_hour']=train['Time_Order_picked'].str.split(':').str[0]
    train['order_picked_min']=train['Time_Order_picked'].str.split(':').str[1]
    train['order_hour']=train['order_hour'].astype(int)
    train['order_min']=train['order_min'].astype(int)
    train['order_picked_hour']=train['order_picked_hour'].astype(int)
    train['order_picked_min']=train['order_picked_min'].astype(int)
    train['increm_order_hour']=np.where(train['order_min']!=60,0,1)
    train['increm_order_picked_hour']=np.where(train['order_picked_min']!=60,0,1)
    train['order_min']=np.where(train['order_min']==60,0,train['order_min'])
    train['order_picked_min']=np.where(train['order_picked_min']==60,0,train['order_picked_min'])
    train['order_hour']=train['increm_order_hour']+train['order_hour']
    train['order_picked_hour']=train['increm_order_picked_hour']+train['order_picked_hour']
    train['order_hour']=np.where(train['order_hour']==24,0,train['order_hour'])
    train['order_picked_hour']=np.where(train['order_picked_hour']==24,0,train['order_picked_hour'])
    train['list_of_order_&_picked']=list(zip(train['order_hour'],train['order_min'],train['order_picked_hour'],
                                             train['order_picked_min']))
    train.drop(columns=['order_picked_hour','order_picked_min','increm_order_hour','order_hour',
                       'increm_order_picked_hour','order_min'],inplace=True)

In [18]:
for_pick_dur(Df)

In [19]:
def dur_btw_order_and_pick(x):
    order_hour=int(x[0])
    order_min=int(x[1])
    picked_hour=int(x[2])
    picked_min=int(x[3])
    if order_hour==picked_hour:
        return picked_min-order_min
    else:
        return 60-order_min+picked_min

In [20]:
Df['dur_btw_order_and_pick']=Df['list_of_order_&_picked'].apply(dur_btw_order_and_pick)

In [ ]:
#Here is the process of creating new column having distance between restaurant and delivery location

In [21]:
Df['res_loc']=list(zip(list(zip(Df['Restaurant_latitude'],Df['Restaurant_longitude'])),list(zip(Df[
    'Delivery_location_latitude'],Df['Delivery_location_longitude']))))



In [22]:
Df['dis_btw_rest_deliv_place']=Df['res_loc'].apply(lambda x :hs.haversine(x[0],x[1]))

In [ ]:
#Now here creating new column having distance between restaurant and delivery location  of those columns which have 
# have not null vlaues

In [23]:
na_lat=na_data[na_data['Restaurant_latitude'].notna()]

In [24]:
na_lat['res_loc']=list(zip(list(zip(na_lat['Restaurant_latitude'],na_lat['Restaurant_longitude'])),
                                list(zip(na_lat[
    'Delivery_location_latitude'],na_lat['Delivery_location_longitude']))))

In [25]:
na_lat['dis_btw_rest_deliv_place']=na_lat['res_loc'].apply(lambda x :hs.haversine(x[0],x[1]))

In [26]:
residual=na_data[na_data['Restaurant_latitude'].isna()]

In [27]:
new=pd.concat([na_lat,residual])

In [ ]:
#now creating new column having duration between oreder and picked  of those columns which have 
# have not null vlaues

In [28]:
ordrna=new[new.Time_Orderd.isna()]

In [29]:
ordr=new[new.Time_Orderd.notna()]

In [30]:
for_pick_dur(ordr)
ordr['dur_btw_order_and_pick']=ordr['list_of_order_&_picked'].apply(dur_btw_order_and_pick)

In [31]:
new_ord=pd.concat([ordrna,ordr])

In [32]:
new_ord['Order_Date']=pd.to_datetime(new_ord.Order_Date)
new_ord['weekday']=new_ord['Order_Date'].dt.weekday

In [33]:
new_ord.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken (min)', 'type',
       'weekday', 'res_loc', 'dis_btw_rest_deliv_place',
       'list_of_order_&_picked', 'dur_btw_order_and_pick'],
      dtype='object')

In [34]:
Df.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken (min)', 'type',
       'weekday', 'list_of_order_&_picked', 'dur_btw_order_and_pick',
       'res_loc', 'dis_btw_rest_deliv_place'],
      dtype='object')

In [35]:
col=['ID','Order_Date','Time_Orderd','Time_Order_picked','list_of_order_&_picked','res_loc','Restaurant_latitude',
     'Restaurant_longitude','Delivery_location_latitude',
    'Delivery_location_longitude',
    'Delivery_person_ID']

In [ ]:
#now concatenating all the not na rows and na rows

In [36]:
new_train=pd.concat([Df,new_ord])

In [37]:
new_train.drop(columns=col,inplace=True)

In [38]:
new_train

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),type,weekday,dur_btw_order_and_pick,dis_btw_rest_deliv_place
0,37.000000,4.900000,0.0,2.0,2,0,0,0.000000,0.0,1.0,24.000000,train,5,15.0,3.025153
1,34.000000,4.500000,1.0,3.0,2,0,1,1.000000,0.0,0.0,33.000000,train,4,5.0,20.183558
2,23.000000,4.400000,2.0,0.0,0,1,0,1.000000,0.0,1.0,26.000000,train,5,15.0,1.552760
3,38.000000,4.700000,0.0,1.0,0,2,0,1.000000,0.0,0.0,21.000000,train,2,10.0,7.790412
4,32.000000,4.600000,3.0,2.0,1,0,1,1.000000,0.0,0.0,30.000000,train,5,15.0,6.210147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11356,30.00,4.50,1.0,1.0,2,1,2,1.00,0.0,0.0,1,test,3,15.0,NaN
11367,27.00,NaN,1.0,3.0,1,0,0,1.00,0.0,0.0,1,test,5,15.0,NaN
11383,39.00,5.00,1.0,2.0,0,3,0,1.00,0.0,0.0,1,test,1,15.0,NaN
11391,26.00,4.80,3.0,0.0,0,1,0,0.00,0.0,1.0,1,test,5,10.0,NaN


In [ ]:
#spliting train data anda test data

In [39]:
org_train=new_train[new_train['type']=='train']
org_test=new_train[new_train['type']=='test']
target=org_train.loc[:,'Time_taken (min)']
org_test.drop(columns=['Time_taken (min)','type'],inplace=True)
org_train.drop(columns=['Time_taken (min)','type'],inplace=True)

In [40]:
org_train

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,weekday,dur_btw_order_and_pick,dis_btw_rest_deliv_place
0,37.000000,4.900000,0.0,2.0,2,0,0,0.000000,0.0,1.0,5,15.0,3.025153
1,34.000000,4.500000,1.0,3.0,2,0,1,1.000000,0.0,0.0,4,5.0,20.183558
2,23.000000,4.400000,2.0,0.0,0,1,0,1.000000,0.0,1.0,5,15.0,1.552760
3,38.000000,4.700000,0.0,1.0,0,2,0,1.000000,0.0,0.0,2,10.0,7.790412
4,32.000000,4.600000,3.0,2.0,1,0,1,1.000000,0.0,0.0,5,15.0,6.210147
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45569,26.00,4.90,4.0,3.0,1,1,1,1.00,0.0,0.0,0,10.0,NaN
45576,34.00,4.20,1.0,1.0,0,1,0,3.00,0.0,1.0,0,5.0,NaN
45577,25.00,4.70,3.0,0.0,0,2,0,1.00,0.0,0.0,1,5.0,NaN
45579,32.00,4.60,1.0,1.0,2,1,1,1.00,0.0,0.0,1,10.0,NaN


In [41]:
org_test

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,weekday,dur_btw_order_and_pick,dis_btw_rest_deliv_place
1,28.000000,4.600000,5.0,3.0,0,0,0,1.000000,0.0,0.0,4,5.0,17.076641
2,23.000000,4.500000,1.0,3.0,0,1,0,1.000000,0.0,0.0,5,10.0,13.406048
3,21.000000,4.800000,4.0,1.0,1,3,1,1.000000,0.0,0.0,2,5.0,13.413663
4,31.000000,4.600000,0.0,1.0,2,1,1,1.000000,0.0,0.0,5,15.0,17.076282
5,26.000000,4.700000,4.0,0.0,0,1,0,1.000000,0.0,0.0,3,10.0,1.543935
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11356,30.00,4.50,1.0,1.0,2,1,2,1.00,0.0,0.0,3,15.0,NaN
11367,27.00,NaN,1.0,3.0,1,0,0,1.00,0.0,0.0,5,15.0,NaN
11383,39.00,5.00,1.0,2.0,0,3,0,1.00,0.0,0.0,1,15.0,NaN
11391,26.00,4.80,3.0,0.0,0,1,0,0.00,0.0,1.0,5,10.0,NaN


In [ ]:
#imputing nan values

In [42]:

k=KNNImputer(n_neighbors=5)
to_train=pd.DataFrame(k.fit_transform(org_train))

In [ ]:
#training the model

In [43]:
from flaml import AutoML
automl_log2 = AutoML() 
tqdm(automl_log2.fit(to_train,target, task="regression",metric='r2',
                     estimator_list =['lgbm'],time_budget=500,early_stop=True,n_splits=10))


[flaml.automl: 09-04 16:03:26] {2444} INFO - task = regression
[flaml.automl: 09-04 16:03:26] {2446} INFO - Data split method: uniform
[flaml.automl: 09-04 16:03:26] {2449} INFO - Evaluation method: cv
[flaml.automl: 09-04 16:03:26] {2568} INFO - Minimizing error metric: 1-r2
[flaml.automl: 09-04 16:03:26] {2708} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 09-04 16:03:26] {3010} INFO - iteration 0, current learner lgbm
[flaml.automl: 09-04 16:03:27] {3143} INFO - Estimated sufficient time budget=10661s. Estimated necessary time budget=11s.
[flaml.automl: 09-04 16:03:27] {3190} INFO -  at 1.6s,	estimator lgbm's best error=0.7855,	best estimator lgbm's best error=0.7855
[flaml.automl: 09-04 16:03:27] {3010} INFO - iteration 1, current learner lgbm
[flaml.automl: 09-04 16:03:28] {3190} INFO -  at 2.6s,	estimator lgbm's best error=0.7855,	best estimator lgbm's best error=0.7855
[flaml.automl: 09-04 16:03:28] {3010} INFO - iteration 2, current learner lgbm
[flaml.autom

In [ ]:
#imputing na values of test data

In [44]:

to_test=pd.DataFrame(k.transform(org_test))

In [ ]:
#prediction test data

In [47]:
pred=automl_log2.predict(to_test)



In [50]:
pred=list(map(float,pred))

In [51]:
submission=pd.DataFrame({'ID':id,'Time_taken (min)':pred})

In [ ]:
#saving to csv file

In [54]:
submission.to_csv('dataset\\sample_submission.csv',index=False)